<a href="https://colab.research.google.com/github/ash-rulz/TextMining/blob/main/TextMiningProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG
This notebook implements a RAG pipeline for answering questions related to the first book in the Harry Potter series - "Harry Potter and the Sorcer's Stone".

The whole process can be summarized as follows:
1. **PDF splitter**: A PDF version of the book is parsed and split into smaller chunks.
2. **Sentence embedding**: These chunks of information is transformed into sentence embeddings. The sentence embedder user for this is *sentence-transformers/all-MiniLM-L6-v2*.
3. **Vector DB**: The embeddings are stored in vector DB. The vector DB used here is FAISS.
4. **Generator**: A generator based on a LLM is created. The generator used here is *google/flan-t5-large*.
5. **Retriever chain**: A retriever chain is then created. The input to a retriever chain will be a question. The question is converted into a sentence embedding which is then compared with the embedding in the vector DB. The k most similar documents are retrieved from the vector DB. These documents are passed as *context* in the custom made prompt template, along with the original question. This prompt is passed to the Generator to get the answer to the question.
6. **Evaluation**: The whole RAG pipeline is evaluated using the RAGAS framework.
7. **Improvements**: We try out different ways to improve the evaluation scores.

In [ ]:
!pip install -q -U langchain pypdf

In [ ]:
#Load the pdf to memory
from langchain.document_loaders import PyPDFLoader
pdfLoader = PyPDFLoader("Book.pdf")
documents = pdfLoader.load()

In [ ]:
#Split the file to chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=10,
    separators=['\n\n', '\n', '(?=>\. )', ' ', ''])
docs = text_splitter.split_documents(documents)
len(docs)

4059

In [ ]:
!pip install -U -q sentence-transformers faiss-gpu

In [ ]:
#Store the documents in a vector store
from langchain.embeddings import HuggingFaceEmbeddings

model_path = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name=model_path,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
#Create vector store
from langchain.vectorstores import FAISS
db = FAISS.from_documents(docs, embeddings)

In [ ]:
#Example of documents retrieved from the vector DB for a question
question = "What is the name of Filch's cat?"
searchDocs = db.similarity_search_with_score(question)
print(searchDocs[0])

(Document(page_content='106Filch owned a cat called Mrs. Norris, a scrawny, dust-colored creature', metadata={'source': 'Book.pdf', 'page': 106}), 0.74705535)


In [ ]:
#Create a generator
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,pipeline
from langchain import HuggingFacePipeline

model_name_flan = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name_flan)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_flan)
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer,max_new_tokens=200)
llm = HuggingFacePipeline(
    pipeline = pipe,
    model_kwargs={"temperature": 0, "max_length": 1000000},
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
#Evidence of hallucination by the T5 model
question = "What is the name of Filch's cat?"
llm_result = llm.invoke(question)
llm_result

'sam'

In [ ]:
#Create a prompt template
from langchain.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

### CONTEXT
{context}

### QUESTION
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
#Create the retriever and the QA chain
base_retriever = db.as_retriever(search_kwargs={"k" : 3})
retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | base_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | llm, "context": itemgetter("context")}
)

In [ ]:
#Evidence of how the RAG improved the result
result = retrieval_augmented_qa_chain.invoke({"question" : question})
result['response']

'Mrs. Norris'

# Evaluation

In [ ]:
!pip install -q datasets tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00


In [ ]:
#Get the ground truth data from QAEval
from datasets import Dataset
#eval_dataset = Dataset.from_csv("QAEval.csv", encoding='latin1', sep = ';')
eval_dataset = Dataset.from_csv("QAEval.csv")
eval_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['question', 'ground_truth'],
    num_rows: 141
})

The QAEval.csv containing the ground truth data is created in the [EvalDataGenerator](https://colab.research.google.com/github/ash-rulz/TextMining/blob/main/EvalDataGenerator.ipynbhttps://) notebook.

In [ ]:
#Format the data into the RAGAS structure
from tqdm import tqdm
import pandas as pd

def create_ragas_dataset(rag_chain, eval_dataset):
  rag_dataset = []
  for row in tqdm(eval_dataset):
    answer = rag_chain.invoke({"question" : row["question"]})
    rag_dataset.append(
        {"question" : row["question"],
         "answer" : answer["response"],
         "contexts" : [context.page_content for context in answer["context"]],
         "ground_truths" : [row["ground_truth"]]
         }
    )
  rag_df = pd.DataFrame(rag_dataset)
  rag_eval_dataset = Dataset.from_pandas(rag_df)
  return rag_eval_dataset

In [ ]:
basic_qa_ragas_dataset = create_ragas_dataset(retrieval_augmented_qa_chain, eval_dataset)
basic_qa_ragas_dataset

100%|██████████| 141/141 [16:45<00:00,  7.13s/it]


Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 141
})

In [ ]:
# Save the dataset to a Parquet file
save_path = '/content/basic_qa_ragas_dataset.parquet'
basic_qa_ragas_dataset.to_pandas().to_parquet(save_path)

In [ ]:
from datasets import Dataset
import pandas as pd

save_path = '/content/basic_qa_ragas_dataset.parquet'
ragas_eval_dataset =  Dataset.from_pandas(pd.read_parquet(save_path))
ragas_eval_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 141
})

In [ ]:
!pip install -q -U ragas openai

RAGAs evaluation needs OpenAI api key.

In [ ]:
import os
import openai
import getpass

open_ai_key = getpass.getpass('Enter your OPENAI API Key')
os.environ['OPENAI_API_KEY'] = open_ai_key

In [ ]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    context_relevancy,
    answer_correctness,
    answer_similarity
)
from ragas import evaluate
eval_result = evaluate(
  ragas_eval_dataset,
  metrics=[
      context_precision,
      faithfulness,
      answer_relevancy,
      context_recall,
      context_relevancy,
      answer_correctness,
      answer_similarity
  ],
)
eval_result

evaluating with [context_precision]


100%|██████████| 10/10 [02:13<00:00, 13.31s/it]


evaluating with [faithfulness]


100%|██████████| 10/10 [02:57<00:00, 17.77s/it]


evaluating with [answer_relevancy]


...

evaluating with [answer_correctness]


100%|██████████| 10/10 [01:34<00:00,  9.49s/it]


evaluating with [answer_similarity]


100%|██████████| 10/10 [00:17<00:00,  1.74s/it]


{'context_precision': 0.2996, 'faithfulness': 0.4255, 'answer_relevancy': 0.6323, 'context_recall': 0.6390, 'context_relevancy': 0.1007, 'answer_correctness': 0.4832, 'answer_similarity': 0.8691}

# Improving Retriever scores

In [ ]:
!pip install -q -U chromadb

In [ ]:
len(documents)

250

In [ ]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=500)

# This text splitter is used to create the child documents
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=embeddings
)

# The storage layer for the parent documents
store = InMemoryStore()

In [ ]:
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [ ]:
retriever.add_documents(documents)

In [ ]:
len(list(store.yield_keys()))

1351

In [ ]:
#Create the QA chain with the Parent Document Retriever
improved_qa_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | llm, "context": itemgetter("context")}
)

#Check if the answer is correct
result = improved_qa_chain.invoke({"question" : question})
result['response']

'Mrs. Norris'

In [ ]:
from datasets import Dataset
import pandas as pd

#Create the RAGAs dataset
improved_ragas_dataset = create_ragas_dataset(improved_qa_chain, eval_dataset)

# Save the dataset to a Parquet file
save_path = '/content/improved_ragas_dataset.parquet'
improved_ragas_dataset.to_pandas().to_parquet(save_path)

100%|██████████| 141/141 [19:06<00:00,  8.13s/it]


Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 141
})

In [ ]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    context_relevancy,
    answer_correctness,
    answer_similarity
)
from ragas import evaluate

#Load the dataset corresponding to the imrpoved retriever
save_path = '/content/improved_ragas_dataset.parquet'
improved_ragas_eval_dataset =  Dataset.from_pandas(pd.read_parquet(save_path))

eval_result = evaluate(
  improved_ragas_eval_dataset,
  metrics=[
      context_precision,
      faithfulness,
      answer_relevancy,
      context_recall,
      context_relevancy,
      answer_correctness,
      answer_similarity
  ],
)
eval_result

evaluating with [context_precision]


100%|██████████| 10/10 [01:06<00:00,  6.62s/it]


evaluating with [faithfulness]


100%|██████████| 10/10 [02:27<00:00, 14.73s/it]


evaluating with [answer_relevancy]


...

evaluating with [answer_correctness]


100%|██████████| 10/10 [00:56<00:00,  5.61s/it]


evaluating with [answer_similarity]


100%|██████████| 10/10 [00:08<00:00,  1.23it/s]


{'context_precision': 0.2187, 'faithfulness': 0.2695, 'answer_relevancy': 0.4153, 'context_recall': 0.5130, 'context_relevancy': 0.1071, 'answer_correctness': 0.3574, 'answer_similarity': 0.8337}

The scores have worsened. This could be due to the parent chunk size being 500. The generator has a context window size of only 512. That means that the instruction, question and the context should come under this limit, but this doesn't. Probably, if there was a better generator model with longer context length, it might have had better results.

# References
The above work is based on a proof-of-concept available in [github location](https://github.com/ash-rulz/RAG/blob/main/RAG_Langchain.ipynb).